In [1]:
%config IPCompleter.greedy=True
import re
import json
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
import requests
from time import time
import glob
from datetime import datetime
import os


es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 360, 'maxsize': 25}])

In [6]:
def recreate_index():
    es.indices.delete(index='myandex')
    es.indices.create(index='myandex', body=settings)
recreate_index()

In [4]:
settings = {
    'mappings': {
        'properties': {
            'content': {
                'type': 'text'
            },
        }
    },
    'settings': {
        'analysis': {
            'analyzer': {
                'russian_complex': {
                    'tokenizer': 'word_longer_2',
                    'filter': [
                        'lowercase',
                        'russian_snow'
                    ]
                }
            },
            'tokenizer': {
                'word_longer_2': {
                    'type': 'pattern',
                    'pattern': '[a-zA-Z_0-9\u0400-\u04FF]{2,}',
                    'group': 0
                }
            },
            'filter': {
                'russian_snow': {
                    'type': 'snowball',
                    'language': 'russian'
                }
            }
        }
    }
}

In [7]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

def es_actions_generator():
    for filename in glob.glob("/Users/aleksandra.orlova/Downloads/byweb_for_course/text1/document*.txt"):
        with open(filename, 'r') as inf:
            doc = json.dumps({'content': inf.read()})
        doc_id = (os.path.splitext(os.path.basename(inf.name))[0])[8:]
        yield create_es_action('myandex', doc_id, doc)

In [8]:
now = datetime.now()
print(now)
for ok, result in parallel_bulk(es, es_actions_generator(), queue_size=400000, thread_count=16, chunk_size=500):
    if not ok:
        print(result)
now = datetime.now()
print(now)

2019-09-23 22:17:40.827500
2019-09-23 22:20:31.871915


In [9]:
es.indices.stats()

{'_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_all': {'primaries': {'docs': {'count': 192338, 'deleted': 0},
   'store': {'size_in_bytes': 2908940333},
   'indexing': {'index_total': 200000,
    'index_time_in_millis': 634310,
    'index_current': 0,
    'index_failed': 0,
    'delete_total': 0,
    'delete_time_in_millis': 0,
    'delete_current': 0,
    'noop_update_total': 0,
    'is_throttled': False,
    'throttle_time_in_millis': 0},
   'get': {'total': 0,
    'time_in_millis': 0,
    'exists_total': 0,
    'exists_time_in_millis': 0,
    'missing_total': 0,
    'missing_time_in_millis': 0,
    'current': 0},
   'search': {'open_contexts': 0,
    'query_total': 0,
    'query_time_in_millis': 0,
    'query_current': 0,
    'fetch_total': 0,
    'fetch_time_in_millis': 0,
    'fetch_current': 0,
    'scroll_total': 0,
    'scroll_time_in_millis': 0,
    'scroll_current': 0,
    'suggest_total': 0,
    'suggest_time_in_millis': 0,
    'suggest_current': 0},
   'merges': 

In [43]:
def search(query, *args):
    return pretty_print_result(es.search(index='myandex', body=query, size=1000), args)
    # note that size set to 20 just because default value is 10 and we know that we have 12 docs and 10 < 12 < 20
                        
def pretty_print_result(search_result, fields=[]):
    # fields is a list of fields names which we want to be printed
    res = search_result['hits']
    print(f'Total documents: {res["total"]["value"]}')
    data = {}
    for hit in res['hits']:
        data[hit['_id']] = 1
    return data
                  
def get_doc_by_id(doc_id):
    return es.get(index='myandex', id=doc_id)['_source']

In [44]:
query = {
    'query': {
        'match_all': {}
    }
}

search(query, ' ')

Total documents: 10000


{'334285': 1,
 '488133': 1,
 '1031806': 1,
 '974701': 1,
 '1162292': 1,
 '374879': 1,
 '1332409': 1,
 '1491847': 1,
 '285894': 1,
 '866477': 1,
 '344762': 1,
 '1463409': 1,
 '536811': 1,
 '1067498': 1,
 '1084703': 1,
 '452429': 1,
 '1112775': 1,
 '857872': 1,
 '387729': 1,
 '1284977': 1,
 '551652': 1,
 '284552': 1,
 '882083': 1,
 '1464366': 1,
 '1058668': 1,
 '365974': 1,
 '923659': 1,
 '475096': 1,
 '895299': 1,
 '1009427': 1,
 '830157': 1,
 '876102': 1,
 '1060491': 1,
 '499758': 1,
 '1102200': 1,
 '1219840': 1,
 '404109': 1,
 '518252': 1,
 '546448': 1,
 '795864': 1,
 '810287': 1,
 '935985': 1,
 '1070682': 1,
 '1516082': 1,
 '1390916': 1,
 '403666': 1,
 '77711': 1,
 '31744': 1,
 '1497588': 1,
 '1462071': 1,
 '1046330': 1,
 '558676': 1,
 '115581': 1,
 '488655': 1,
 '27126': 1,
 '386351': 1,
 '282645': 1,
 '314633': 1,
 '463192': 1,
 '1335400': 1,
 '480209': 1,
 '894187': 1,
 '1082414': 1,
 '1353785': 1,
 '593261': 1,
 '556883': 1,
 '1292873': 1,
 '497775': 1,
 '501703': 1,
 '1226468': 

In [45]:
from bs4 import BeautifulSoup
import time

file = f"/Users/aleksandra.orlova/Downloads/web2008_adhoc.xml"
f = open(file, "rb")
contents = f.read()
soup = BeautifulSoup(contents, 'xml')

spend_time = 0
table = {}
for task in soup.find_all('task'):
    text = task.querytext.text
    timestamp = int(time.time())
    data = search(create_query(text), ' ')
    spend_time += int(time.time()) - timestamp
    table[task['id']] = data
    
print(spend_time)

Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 6112
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 0
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 1371
Total documents: 10000
Total documents: 2474
Total documents: 7450
Total documents: 10000
Total documents: 73
Total documents: 10000
Total documents: 10000

Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 796
Total documents: 10000
Total documents: 10000
Total documents: 962
Total documents: 8027
Total documents: 10000
Total documents: 0
Total documents: 6298
Total documents: 55
Total documents: 9292
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 1836
Total documents: 1495
Total documents: 101
Total documents: 106
Total documents: 776
Total documents: 7209
Total documents: 7233
Total documents: 10000
Total documents: 482
Total documents: 10000
Total documents: 1570
Total documents: 1878
Total documents: 10000
Total documents: 10000
Total documents: 3000
Total documents: 7779
Total documents: 3155
Total documents: 3534
Total documents: 4969
Total documents: 7435
Total documents: 384
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 6575
Total documents: 10000
Total documents: 6575
Total documents: 10000
Tot

Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 45
Total documents: 1090
Total documents: 245
Total documents: 10000
Total documents: 10000
Total documents: 400
Total documents: 10000
Total documents: 217
Total documents: 7073
Total documents: 10000
Total documents: 3121
Total documents: 10000
Total documents: 10000
Total documents: 5664
Total documents: 10000
Total documents: 126
Total documents: 10000
Total documents: 2444
Total documents: 158
Total documents: 10000
Total documents: 10000
Total documents: 2057
Total documents: 10000
Total documents: 10000
Total documents: 1531
Total documents: 0
Total documents: 950
Total documents: 313
Total documents: 128
Total documents: 1856
Total documents: 1790
Total documents: 3489
Total documents: 10000
Total documents: 3712
Total documents: 4666
Total documents: 100
Total documents: 357
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 4392
Total documents: 993
Total d

Total documents: 10000
Total documents: 10000
Total documents: 19
Total documents: 10
Total documents: 313
Total documents: 165
Total documents: 10000
Total documents: 186
Total documents: 9176
Total documents: 0
Total documents: 3690
Total documents: 205
Total documents: 10000
Total documents: 4795
Total documents: 29
Total documents: 10000
Total documents: 10000
Total documents: 188
Total documents: 10000
Total documents: 10000
Total documents: 9989
Total documents: 10000
Total documents: 3242
Total documents: 10000
Total documents: 3894
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 615
Total documents: 10000
Total documents: 1228
Total documents: 10000
Total documents: 3124
Total documents: 10000
Total documents: 4310
Total documents: 10000
Total documents: 36
Total documents: 312
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 7882
Total documents: 9
Total doc

Total documents: 7517
Total documents: 6272
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 6292
Total documents: 6352
Total documents: 6949
Total documents: 5589
Total documents: 1054
Total documents: 4
Total documents: 475
Total documents: 763
Total documents: 8133
Total documents: 1127
Total documents: 10000
Total documents: 10000
Total documents: 839
Total documents: 10000
Total documents: 10000
Total documents: 109
Total documents: 5075
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 488
Total documents: 654
Total documents: 644
Total documents: 8682
Total documents: 1890
Total documents: 93
Total documents: 285
Total documents: 10000
Total documents: 9699
Total documents: 113
Total documents: 10000
Total documents: 979
Total documents: 10000
Total documents: 8110
Total documents: 3614
Total documents: 8780
Total documents: 10000
Total documents: 40
Total documents: 1387
Total docume

...

Total documents: 10000
Total documents: 0
Total documents: 10000
Total documents: 3382
Total documents: 1247
Total documents: 9203
Total documents: 23
Total documents: 8344
Total documents: 2495
Total documents: 10000
Total documents: 8105
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 164
Total documents: 10000
Total documents: 8
Total documents: 6636
Total documents: 10000
Total documents: 5792
Total documents: 10000
Total documents: 10000
Total documents: 307
Total documents: 7804
Total documents: 0
Total documents: 3139
Total documents: 6814
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 109
Total documents: 4552
Total documents: 10000
Total documents: 5182
Total documents: 1196
Total documents: 10000
Total documents: 1493
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 2471
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 0
Total 

Total documents: 224
Total documents: 2097
Total documents: 10000
Total documents: 59
Total documents: 10000
Total documents: 11
Total documents: 10000
Total documents: 7897
Total documents: 10000
Total documents: 1975
Total documents: 419
Total documents: 10000
Total documents: 7740
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 92
Total documents: 10000
Total documents: 7432
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 5849
Total documents: 69
Total documents: 10000
Total documents: 10000
Total documents: 9
Total documents: 10000
Total documents: 4668
Total documents: 1
Total documents: 2691
Total documents: 10000
Total documents: 1805
Total documents: 10000
Total documents: 10000
Total documents: 307
Total documents: 5617
Total documents: 10000
Total documents: 10000
Total documents: 1116
Total documents: 10000
To

Total documents: 10000
Total documents: 3
Total documents: 0
Total documents: 10000
Total documents: 10000
Total documents: 178
Total documents: 833
Total documents: 10000
Total documents: 10000
Total documents: 215
Total documents: 10000
Total documents: 10000
Total documents: 4377
Total documents: 10000
Total documents: 0
Total documents: 0
Total documents: 0
Total documents: 10000
Total documents: 10000
Total documents: 139
Total documents: 2235
Total documents: 10000
Total documents: 5615
Total documents: 10000
Total documents: 160
Total documents: 0
Total documents: 0
Total documents: 10000
Total documents: 10000
Total documents: 401
Total documents: 10000
Total documents: 10000
Total documents: 0
Total documents: 854
Total documents: 119
Total documents: 0
Total documents: 10000
Total documents: 0
Total documents: 8834
Total documents: 0
Total documents: 0
Total documents: 160
Total documents: 10000
Total documents: 4
Total documents: 2226
Total documents: 2670
Total documents: 8

Total documents: 10000
Total documents: 0
Total documents: 10000
Total documents: 0
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 431
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 5780
Total documents: 5190
Total documents: 1161
Total documents: 10000
Total documents: 0
Total documents: 284
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 2301
Total documents: 10000
Total documents: 10000
Total documents: 2991
Total documents: 10000
Total documents: 10000
Total documents: 2572
Total documents: 141
Total documents: 10000
Total documents: 6768
Total documents: 7
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 0
Total documents: 80
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 3695
Total documents: 10000
Total documents: 7469
Tot

Total documents: 10000
Total documents: 3
Total documents: 3296
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 1
Total documents: 738
Total documents: 10000
Total documents: 160
Total documents: 8813
Total documents: 4933
Total documents: 10000
Total documents: 2084
Total documents: 3155
Total documents: 10000
Total documents: 4631
Total documents: 10000
Total documents: 2
Total documents: 531
Total documents: 4630
Total documents: 1455
Total documents: 10000
Total documents: 10000
Total documents: 73
Total documents: 2999
Total documents: 10000
Total documents: 320
Total documents: 1
Total documents: 3476
Total documents: 7
Total documents: 10000
Total documents: 362
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 6858
Total documents: 9583
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total documents: 10000
Total docu

In [32]:
def create_query(text):
    query = {
    'query': {
        'bool': {
            'should': [
                {
                    'match': {
                            'content': text
                        }
                    }
                ]
            }
        }
    }
    return query

In [34]:
query = create_query('pda')
search(create_query(text), ' ')

Total documents: 7661
965565
391583
471208
574718
686767
416312
452927
1281669
1003770
46177
1010087
881777
247964
713482
320627
1338766
881800
1285234
46178
247936
712045
1287341
953832
575770
1070070
69399
421945
1074933
1390972
679871
775565
575775
1388882
928599
910356
679853
999428
1003035
965566
569998
247945
679875
1070082
775581
1443907
1357738
681809
1412143
989751
247959
1070072
712190
712298
50252
1462136
575759
421936
247938
247954
783207
657060
450702
320309
1435573
951947
450883
44156
919627
46195
575769
769805
42135
91273
1080545
1012007
473214
320631
1003046
773452
386903
1267734
888219
597296
390359
876080
1147052
921344
1152969
575762
919631
287640
1271612
46299
1478374
320441
1010637
1287337
247935
1106924
1156539
779438
682091
1400323
320870
877213
1083842
1080539
575786
46207
411256
783201
73930
247943
1263675
1010089
572129
572841
1221592
1003057
863062
913277
1443969
275171
291928
575767
1022984
1396996
1233191
952056
1065964
1242644
470734
712192
271415
473098
5

In [53]:
file = f"/Users/aleksandra.orlova/Downloads/or_relevant-minus_table.xml"

def get_relevance():
    relevance = {}
    with open(file,'r', encoding="cp1251") as src:
        raw_xml = src.read()
        soup = BeautifulSoup(raw_xml)
        
        for task in soup.find_all('task'):
            documents = task.find_all('document') 
            vital = set()
            for doc in documents:
                if doc['relevance'] == 'vital':
                    vital.add(doc['id'])
            if vital:
                relevance[task['id']] = vital
    return relevance
relevance = get_relevance()

In [55]:
file = f"/Users/aleksandra.orlova/Downloads/web2008_adhoc.xml"

def get_query_ids():
    f = open(file, "rb")
    contents = f.read()
    soup = BeautifulSoup(contents, 'xml')
    queries = []
    for task in soup.find_all('task'):
        queries.append(task['id'])
    return queries

queries = get_query_ids()

In [94]:
K = 20

def get_relevant_for_k(res, relevant, k):
    return sum([1 if doc in relevant else 0 for doc in res[:k]])        

def analyze_results():
    Q = 0
    qpK, qrK, qR_average, qmapK = 0, 0, 0, 0
    qR = []
    for task_id in queries:
        results = [k for k in table[task_id]][:K]
        if task_id not in relevance:
            continue
        Q += 1
        cur_relevant = len(relevance[task_id])
        qpK += get_relevant_for_k(results, relevance[task_id], K) / K 
        qrK += get_relevant_for_k(results, relevance[task_id], K) / cur_relevant
        qR.append(get_relevant_for_k(results, relevance[task_id], cur_relevant) / cur_relevant)
        qR_average += qR[-1]
        mapK = 0
        for k in range(1, K + 1):
            mapK += get_relevant_for_k(results, relevance[task_id], k) / k
        mapK /= K
        qmapK += mapK
    print(f"p@20 {qpK / Q}")
    print(f"r@20 {qrK / Q}")
    print(f"R-precision {qR_average / Q}")
    print(f"Average MAP@20 {qmapK / Q}")        

In [95]:
analyze_results()

p@20 0.33989898989899003
r@20 0.22369933258553276
R-precision 0.1874032825482782
Average MAP@20 0.402821758439394


In [96]:
spend_time

18465